# Pipeline of Graph Retrieval

## Initialization

In [1]:
import logging
import os
import gradio as gr
import nest_asyncio
from dotenv import load_dotenv

In [2]:
from ogmyrag.my_logging import configure_logger
from ogmyrag.base import MongoStorageConfig, Neo4jStorageConfig, PineconeStorageConfig
from ogmyrag.graph_retrieval import GraphRetrievalSystem

In [3]:
# Patch event loop to support re-entry in Jupyter
nest_asyncio.apply()

In [4]:
# Setup logging
graph_retrieval_logger = configure_logger(name='graph_retrieval',log_level=logging.DEBUG, log_file='logs/graph_retrieval.log')

In [5]:
# Load environment variables
load_dotenv(override=True)

mongo_db_uri = os.getenv("MONGO_DB_URI","")

openai_api_key = os.getenv("OPENAI_API_KEY","")

pinecone_api_key = os.getenv("PINECONE_API_KEY","")
pinecone_environment = os.getenv("PINECONE_ENVIRONMENT","")
pinecone_cloud = os.getenv("PINECONE_CLOUD","")
pinecone_metric = os.getenv("PINECONE_METRIC", "")
pinecone_dimensions = os.getenv("PINECONE_DIMENSIONS")

neo4j_uri = os.getenv("NEO4J_URI","")
neo4j_username = os.getenv("NEO4J_USERNAME","")
neo4j_password = os.getenv("NEO4J_PASSWORD","")

if not mongo_db_uri:
    graph_retrieval_logger.error("Please set the MONGO_DB_URI environment variable.")

if not openai_api_key:
    graph_retrieval_logger.error("Please set the OPENAI_API_KEY environment variable.")

if not pinecone_api_key or not pinecone_environment or not pinecone_cloud or not pinecone_metric or not pinecone_dimensions:
    graph_retrieval_logger.error("Please set the PINECONE_API_KEY, PINECONE_ENVIRONMENT, PINECONE_CLOUD, PINECONE_METRIC, and PINECONE_DIMENSIONS environment variables.")

if not neo4j_uri or not neo4j_username or not neo4j_password:
    graph_retrieval_logger.error("Please set the NE04J_URI, NE04J_USERNAME, and NE04J_PASSWORD environment variables.")

## Initialize Graph Retrieval System

In [6]:
ontology_config: MongoStorageConfig = {
    "connection_uri": mongo_db_uri,
    "database_name": "ogmyrag",
    "collection_name": "ontology_v2",
}

graphdb_config: Neo4jStorageConfig = {
    "uri": neo4j_uri,
    "user": neo4j_username,
    "password": neo4j_password,
}

entity_vector_config: PineconeStorageConfig = {
    "index_name": "ogmyrag",
    "pinecone_api_key": pinecone_api_key,
    "pinecone_environment": pinecone_environment,
    "pinecone_cloud": pinecone_cloud,
    "pinecone_metric": pinecone_metric,
    "pinecone_dimensions": pinecone_dimensions,
    "openai_api_key": openai_api_key
}
    

In [7]:
try:
    graph_system = GraphRetrievalSystem(
        ontology_config=ontology_config,
        entity_vector_config=entity_vector_config,
        graphdb_config=graphdb_config,
    )
except Exception as e:
    graph_retrieval_logger.error(
        f"GraphRetrievalSystem\nError while creating graph retrieval system: {e}"
    )

In [9]:
response = await graph_system.query_from_graph("Who are associated with autocount?")
graph_retrieval_logger.info(response)

2025-07-24 13:48:35,917 - graph_retrieval - INFO - GraphRetrievalSystem
Preparing ontology...
2025-07-24 13:48:36,037 - graph_retrieval - INFO - GraphRetrievalSystem
Preparing ontology...
2025-07-24 13:48:36,103 - graph_retrieval - INFO - UserRequest2QueryAgent is called
2025-07-24 13:48:49,898 - graph_retrieval - INFO - UserRequest2QueryAgent
User request to retrieval query response details:
{
  "id": "resp_6881c93497dc8198b336ee9f29896d9a013c49dbcaf76c75",
  "created_at": 1753336116.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "o4-mini-2025-04-16",
  "object": "response",
  "output": [
    {
      "id": "rs_6881c935cbe081988822cbe2537d6deb013c49dbcaf76c75",
      "summary": [],
      "type": "reasoning"
    },
    {
      "id": "msg_6881c93fc3008198a2ed4bab149b4584013c49dbcaf76c75",
      "content": [
        {
          "annotations": [],
          "text": "{\"user_query\":\"Who are associated with autocount?\",\"generated_qu

In [ ]:
# chat_interface = gr.ChatInterface(
#     fn=graph_system.get_response_from_user2requestagent,
#     title="Ontology-Aware Graph Retrieval",
#     chatbot=gr.Chatbot(),
#     textbox=gr.Textbox(placeholder="Ask a company-related question..."),
#     type="text", 
# )

# chat_interface.launch()